In [1]:
import json # will be needed for saving preprocessing details
import numpy as np # for data manipulation
import pandas as pd # for data manipulation
from sklearn.model_selection import train_test_split # will be used for data split
from sklearn.preprocessing import LabelEncoder # for preprocessing
from sklearn.ensemble import RandomForestClassifier # for training the algorithm
from sklearn.ensemble import ExtraTreesClassifier # for training the algorithm
import joblib # for saving algorithm and preprocessing objects
import sqlite3
from geopy.geocoders import Nominatim

In [2]:
conn = sqlite3.connect("../../db.sqlite3")
cur = conn.cursor()
cur.execute('''select * from sqlite_master''')
l = cur.fetchall()
#we will populate a list with the names of your tables
tbl_name_list = []
for sql_type, sql_name, tbl_name, rootpage, sql in l:
    if sql_type == 'table':
        tbl_name_list.append(sql_name)
df_account = pd.read_sql_query("SELECT * FROM account_account where id=5", conn).drop(
    ["password","is_admin","is_staff","is_superuser","is_active","email","username"], axis=1
)
df_account

,id,date_joined,last_login,birthDate,city,country,postcode,streetName,streetNumber
0,5,2020-11-14 16:42:23.882927,2021-01-02 20:23:43.578461,2020-12-07 18:53:39.744654,Nancy,France,54000,sName,sNumber


In [3]:

df = df_account
df_searched = pd.read_sql_query("SELECT * FROM personal_search_input where user_id=5", conn)
df_searched = df_searched.sort_values(by=['date_search'], ascending=False).drop_duplicates(subset=['user_id'])
df_searched

,id,user_id,Text,date_search
4,8,5,five,2021-01-13 15:12:26.250992


In [4]:
df = df.merge(df_searched.drop(['id'],axis=1),left_on="id",right_on="user_id",how="left")
df


,id,date_joined,last_login,birthDate,city,country,postcode,streetName,streetNumber,user_id,Text,date_search
0,5,2020-11-14 16:42:23.882927,2021-01-02 20:23:43.578461,2020-12-07 18:53:39.744654,Nancy,France,54000,sName,sNumber,5,five,2021-01-13 15:12:26.250992


In [5]:

df['date_joined'] = pd.to_datetime(df['date_joined'])
df['last_login'] = pd.to_datetime(df['last_login'])
df['birthDate'] = pd.to_datetime(df['birthDate'])
df['date_search'] = pd.to_datetime(df['date_search'])
df

,id,date_joined,last_login,birthDate,city,country,postcode,streetName,streetNumber,user_id,Text,date_search
0,5,2020-11-14 16:42:23.882927,2021-01-02 20:23:43.578461,2020-12-07 18:53:39.744654,Nancy,France,54000,sName,sNumber,5,five,2021-01-13 15:12:26.250992


In [6]:
df['location'] = df.streetNumber.map(str) + " , " + df.streetName + " , " + df.postcode + " , " + df.city + " , " + df.country
locator = Nominatim(user_agent="myGeocoder")
df['locatioan'] = locator.geocode(df['location'][0])
df['latitude'] = None
df['longitude'] = None
for i in range(len(df)):
    #print("{} , {}".format(i,df['location'][i]))

    if locator.geocode(df['location'][i]) == None:
        #print("{} , {}".format(i,df['location'][i].split(', ', 1)[1]))
        df['location'][i] = df['location'][i].split(', ', 1)[1]
        if locator.geocode(df['location'][i]) == None:
            df['location'][i] = df['location'][i].split(', ', 1)[1]
            if locator.geocode(df['location'][i]) == None:
                print()
            else:
                print("found 2 {}".format(i))
                location = locator.geocode(df['location'][i])
                print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
                df['latitude'][i] = location.latitude
                df['longitude'][i] = location.longitude
        else:
            print("found 1 {}".format(i))
            location = locator.geocode(df['location'][i])
            print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
            df['latitude'][i] = location.latitude
            df['longitude'][i] = location.longitude
    else:
        print("found 0 {}".format(i))
        location = locator.geocode(df['location'][i])
        print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
        df['latitude'][i] = location.latitude
        df['longitude'][i] = location.longitude
df = df.drop(["city","country","streetName","streetNumber","location","locatioan"],axis=1)
df.dtypes

D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


found 2 0
Latitude = 48.69127075918049, Longitude = 6.181005123242174


D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


id                      int64
date_joined    datetime64[ns]
last_login     datetime64[ns]
birthDate      datetime64[ns]
postcode               object
user_id                 int64
Text                   object
date_search    datetime64[ns]
latitude               object
longitude              object
dtype: object

In [7]:
df["date_joined_year"] = df["date_joined"].dt.year
df["date_joined_month"] = df["date_joined"].dt.month
df["date_joined_week"] = df["date_joined"].dt.week
df["date_joined_day"] = df["date_joined"].dt.day
df["date_joined_hour"] = df["date_joined"].dt.hour
df["date_joined_minute"] = df["date_joined"].dt.minute
df["date_joined_dayofweek"] = df["date_joined"].dt.dayofweek

df["last_login_year"] = df["last_login"].dt.year
df["last_login_month"] = df["last_login"].dt.month
df["last_login_week"] = df["last_login"].dt.week
df["last_login_day"] = df["last_login"].dt.day
df["last_login_hour"] = df["last_login"].dt.hour
df["last_login_minute"] = df["last_login"].dt.minute
df["last_login_dayofweek"] = df["last_login"].dt.dayofweek

df["birthDate_year"] = df["birthDate"].dt.year
df["birthDate_month"] = df["birthDate"].dt.month
df["birthDate_week"] = df["birthDate"].dt.week
df["birthDate_day"] = df["birthDate"].dt.day
df["birthDate_hour"] = df["birthDate"].dt.hour
df["birthDate_minute"] = df["birthDate"].dt.minute
df["birthDate_dayofweek"] = df["birthDate"].dt.dayofweek

df["date_search_year"] = df["date_search"].dt.year
df["date_search_month"] = df["date_search"].dt.month
df["date_search_week"] = df["date_search"].dt.week
df["date_search_day"] = df["date_search"].dt.day
df["date_search_hour"] = df["date_search"].dt.hour
df["date_search_minute"] = df["date_search"].dt.minute
df["date_search_dayofweek"] = df["date_search"].dt.dayofweek
df = df.drop(['date_joined','last_login','birthDate','date_search','id','user_id'],axis=1)
df

,postcode,Text,latitude,longitude,date_joined_year,date_joined_month,date_joined_week,date_joined_day,date_joined_hour,date_joined_minute,...,birthDate_hour,birthDate_minute,birthDate_dayofweek,date_search_year,date_search_month,date_search_week,date_search_day,date_search_hour,date_search_minute,date_search_dayofweek
0,54000,five,48.6913,6.18101,2020,11,46,14,16,42,...,18,53,0,2021,1,2,13,15,12,2
